In [4]:
from dask import dataframe
import pandas as pd

In [6]:
p1 = pd.DataFrame({"a":[1,2,3,4],"b":["one","two","three","four"],"c":[5,6,7,8]}).set_index("a")
p2 = pd.DataFrame({"a":[1,2,3,4,5,6],"e":["one","two","three","four","five","six"],"f":[5,6,7,8,9,10]}).set_index("a")
d1 = dataframe.from_pandas(p1,1)
d2 = dataframe.from_pandas(p2,1)

In [15]:
# default is iner join
print(d1.merge(d2).compute())
print(d1.merge(d2).compute())

       b  c      e  f
a                    
1    one  5    one  5
2    two  6    two  6
3  three  7  three  7
4   four  8   four  8
       b  c      e  f
a                    
1    one  5    one  5
2    two  6    two  6
3  three  7  three  7
4   four  8   four  8


In [18]:
# left join
d2.merge(d1, how="left").compute()

,e,f,b,c
a,,,,
1,one,5,one,5.0
2,two,6,two,6.0
3,three,7,three,7.0
4,four,8,four,8.0
5,five,9,NaN,NaN
6,six,10,NaN,NaN


In [23]:
# default is left join
d1.merge(d2, left_on=["b"],right_on=["e"], how="left").compute()

,b,c,e,f
0,one,5,one,5
1,two,6,two,6
2,three,7,three,7
3,four,8,four,8


In [32]:
# default is left join
import pytest
with pytest.raises(ValueError):
    d1.merge(d2,right_index=True, left_index=True, left_on=["b"],right_on=["e"], how="left").compute()
d1.merge(d2,right_index=True, left_index=True,  how="left").compute()

,b,c,e,f
a,,,,
1,one,5,one,5
2,two,6,two,6
3,three,7,three,7
4,four,8,four,8


In [34]:
# do merges keep indexes?, try indexing the string column, which is less standard
p1 = pd.DataFrame({"a":[1,2,3,4],"b":["one","two","three","four"],"c":[5,6,7,8]}).set_index("b")
p2 = pd.DataFrame({"a":[1,2,3,4,5,6],"e":["one","two","three","four","five","six"],"f":[5,6,7,8,9,10]}).set_index("e")
d1 = dataframe.from_pandas(p1,1)
d2 = dataframe.from_pandas(p2,1)

In [35]:
d1.merge(d2,right_index=True, left_index=True,   how="left").compute()

,a_x,c,a_y,f
b,,,,
four,4,8,4,8
one,1,5,1,5
three,3,7,3,7
two,2,6,2,6


In [37]:
# merging on non index columns disappears the indexes
d1.merge(d2, on="a",  how="left").compute()

,a,c,f
0,4,8,8
1,1,5,5
2,3,7,7
3,2,6,6


In [38]:
# mixing merge with index and non index
p1 = pd.DataFrame({"a":[1,2,3,4],"b":["one","two","three","four"],"c":[5,6,7,8]}).set_index("b")
p2 = pd.DataFrame({"a":[1,2,3,4,5,6],"e":["one","two","three","four","five","six"],"f":[5,6,7,8,9,10]}).set_index("a")
d1 = dataframe.from_pandas(p1,1)
d2 = dataframe.from_pandas(p2,1)

In [39]:
# using one index, but refering two both columns with on, index is reset by default
d1.merge(d2, left_on="a", right_on="a", how="left").compute()

,a,c,e,f
0,4,8,four,8
1,1,5,one,5
2,3,7,three,7
3,2,6,two,6


In [40]:
# using one index, refering the index with right_index, index on left df is kept
d1.merge(d2, left_on="a", right_index=True, how="left").compute()

,a,c,e,f
b,,,,
four,4,8,four,8
one,1,5,one,5
three,3,7,three,7
two,2,6,two,6


In [41]:
# stack overflow 
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'], 
                 'key2': ['K0', 'K1', 'K0', 'K1'],
                 'A': ['A0', 'A1', 'A2', 'A3'],
                 'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3'],
                      'E': [1,2,3,4]})

In [42]:
pd.merge(left, right, left_on=['key2', 'key1'], right_on=['key1', 'key2'], how='outer', indicator=True, left_index=True)

,key1_x,key2_x,A,B,key1_y,key2_y,C,D,E,_merge
0,K0,K0,A0,B0,K0,K0,C0,D0,1.0,both
1,K0,K1,A1,B1,K1,K0,C1,D1,2.0,both
2,K0,K1,A1,B1,K1,K0,C2,D2,3.0,both
3,K1,K0,A2,B2,NaN,NaN,NaN,NaN,NaN,left_only
3,K2,K1,A3,B3,NaN,NaN,NaN,NaN,NaN,left_only
3,NaN,NaN,NaN,NaN,K2,K0,C3,D3,4.0,right_only


In [44]:
pd.merge(left, right,  on=['key1', 'key2'],how='outer', validate = 'one_to_many', indicator=True, left_index = True, right_index = True)

,key1,key2,A,B,C,D,E,_merge
0,K0,K0,A0,B0,C0,D0,1,both
1,K0,K1,A1,B1,C1,D1,2,both
2,K1,K0,A2,B2,C2,D2,3,both
3,K2,K1,A3,B3,C3,D3,4,both


In [45]:
pd.merge(left, right,how='outer', validate = 'one_to_many', indicator=True, left_index = True, right_index = True)

,key1_x,key2_x,A,B,key1_y,key2_y,C,D,E,_merge
0,K0,K0,A0,B0,K0,K0,C0,D0,1,both
1,K0,K1,A1,B1,K1,K0,C1,D1,2,both
2,K1,K0,A2,B2,K1,K0,C2,D2,3,both
3,K2,K1,A3,B3,K2,K0,C3,D3,4,both


In [48]:
#Column-Column Merge: Use left_on, right_on and how.

#Example:

# Gives same answer
pd.merge(left, right, left_on=['key2', 'key1'], right_on=['key1', 'key2'], how = 'outer')
pd.merge(left, right, on=['key1', 'key2'], how='outer', indicator=True)
#Index-Index Merge: Set left_index and right_index to True or use on and use how.

#Example:

pd.merge(left, right, how = 'inner', right_index = True, left_index = True)
# If you make matching unique multi-indexes for both data frames you can do
# pd.merge(left, right, how = 'inner', on = ['indexname1', 'indexname2'])
# In your data frames, you're keys duplicate values so you can't do this
# In general, a column with duplicate values does not make a good key
#Column-Index Merge: Use left_on + right_index or left_index + right_on and how.

#Note: Both the values in index and left_on must match. If you're index is a integer and you're left_on is a string, you get error. Also, number of indexing levels must match.

#Example:

# If how not specified, inner join is used
pd.merge(left, right, right_on=['E'], left_index = True, how = 'outer')  

# Gives error because left_on is string and right_index is integer
with pytest.raises(ValueError):
    pd.merge(left, right, left_on=['key1'], right_index = True, how = 'outer')

# This gave you error because left_on has indexing level of 2 but right_index only has indexing level of 1.
with pytest.raises(ValueError):
    pd.merge(left, right, left_on=['key2', 'key1'], right_on=['key1', 'key2'], how='outer', indicator=True, right_index=True)
#You kind of mix up the different types of merges which gave weird results. If you can't see how the merging is going to happen conceptually, chances are a computer isn't going to do any better.